# Exercise 8: Finding Similar Items

### 1. [10 points] Since the reviews are relatively short pieces of text, we shall utilize a shingling strategy outlined in section 3.2.4 Shingles Built from Words and Example 3.5 on the same page. Write a shingling program that takes a review text and returns a set of shingles.

In [2]:
import pandas as pd

MIN_REVIEWS_PER_ITEM = 25
df = pd.read_csv('reviews.csv')

# Drop rows with no review text
df = df.dropna(subset=['Review Text'])

# Filter Clothing IDs with at least MIN_REVIEWS_PER_ITEM reviews
filtered_ids = df['Clothing ID'].value_counts()
valid_ids = filtered_ids[filtered_ids >= MIN_REVIEWS_PER_ITEM].index
df_filtered = df[df['Clothing ID'].isin(valid_ids)].copy()

print(f"Filtered reviews: {len(df_filtered)}")


Filtered reviews: 19110


The below code takes in the reviews that meet the 25 word minimum criteria and returns 5-word shingles. An example is shown for how it works for one of them. 

In [ ]:
def shingle_review(review, k=5):
    """Returns a set of k-word shingles from a review text."""
    words = review.lower().split()
    shingles = set()
    for i in range(len(words) - k + 1):
        shingle = ' '.join(words[i:i + k])
        shingles.add(shingle)
    return shingles

# Example output of one review's 5-word shingles
example_review = df_filtered['Review Text'].iloc[0]
print(shingle_review(example_review))


{"love this dress! it's sooo", "store, and i'm glad i", 'the knee. would definitely be', 'happened to find it in', 'did bc i never would', "a store, and i'm glad", 'below the knee. would definitely', 'hits just a little below', 'be a true midi on', 'on me- hits just a', 'and am 5\'8". i love', 'a petite and am 5\'8".', 'the length on me- hits', 'knee. would definitely be a', 'i never would have ordered', 'to find it in a', "in a store, and i'm", 'am 5\'8". i love the', 'petite. i bought a petite', 'bc i never would have', 'true midi on someone who', 'i happened to find it', 'i bought a petite and', 'a true midi on someone', 'midi on someone who is', 'i love the length on', 'definitely be a true midi', "it's sooo pretty. i happened", "and i'm glad i did", 'bought a petite and am', 'love the length on me-', 'petite and am 5\'8". i', "bc it's petite. i bought", "it online bc it's petite.", "online bc it's petite. i", "it's petite. i bought a", "i'm glad i did bc", 'me- hits just a little'

### 2. [15 points] Write a minhashing program to calculate minhash values for each review text, using a provided set of 200 hash functions, (each function expressed as a lambda). Time the creation of minhash values for processing all review texts.

The below code minhashes values for each review text using s set of 200 hash function. A very large prime number was found that is larger than next_prime. 

In [4]:
import random
import hashlib
import time

NUM_HASHES = 200
MAX_SHINGLE_ID = 2**32 - 1
next_prime = 4294967311  # a large prime > MAX_SHINGLE_ID

# Generate hash functions
hash_functions = []
for _ in range(NUM_HASHES):
    a = random.randint(1, next_prime - 1)
    b = random.randint(0, next_prime - 1)
    hash_functions.append(lambda x, a=a, b=b: (a * x + b) % next_prime)

def hash_shingle(shingle):
    return int(hashlib.md5(shingle.encode('utf-8')).hexdigest(), 16) % MAX_SHINGLE_ID

def compute_minhash_signature(shingle_set, hash_funcs):
    sig = []
    for h in hash_funcs:
        min_val = float('inf')
        for shingle in shingle_set:
            x = hash_shingle(shingle)
            hx = h(x)
            if hx < min_val:
                min_val = hx
        sig.append(min_val)
    return sig

The code below is used to time the shingling of every review in the csv file. 

In [5]:
start = time.time()
signatures = []

for review in df_filtered['Review Text']:
    shingles = shingle_review(review)
    sig = compute_minhash_signature(shingles, hash_functions)
    signatures.append(sig)

end = time.time()
print(f"Time taken to compute minhash signatures: {end - start:.2f} seconds")


Time taken to compute minhash signatures: 193.40 seconds


When being run on my laptop, the time taken to complete the signatureds was 607.66 seconds, or around 10 minutes and 13 seconds. When being run on my pc (the output above), the time taken was 193.40 seconds. 

### 3. [10 points] An alternative to calculating minhashes from 200 hash functions is to use one hash function and 199 cheap hash functions derived from it, as outlined in this SO post. Time the creation of minhash values for processing all review texts using this method.

The below code shows this alternative method that uses one hash function and 199 cheap hash functions, or XOR-transformed hashes

In [6]:
import numpy as np
import hashlib
import time

NUM_HASHES = 200
MAX_SHINGLE_ID = 2**32 - 1

# Generate 199 random 32-bit integers for XOR masking
xor_masks = np.random.randint(0, MAX_SHINGLE_ID, size=NUM_HASHES - 1, dtype=np.uint32)

def fast_hash_200(shingle):
    """Generate 200 hash values using 1 good hash and 199 cheap XORs."""
    # Base hash: 32-bit from MD5
    base_hash = int(hashlib.md5(shingle.encode()).hexdigest(), 16) & 0xFFFFFFFF
    hashes = [base_hash]
    for mask in xor_masks:
        hashes.append(base_hash ^ mask)
    return hashes

def compute_minhash_signature_fast(shingle_set):
    sig = [float('inf')] * NUM_HASHES
    for shingle in shingle_set:
        h_values = fast_hash_200(shingle)
        for i in range(NUM_HASHES):
            if h_values[i] < sig[i]:
                sig[i] = h_values[i]
    return sig


In [7]:
start = time.time()
signatures_fast = []

for review in df_filtered['Review Text']:
    shingles = shingle_review(review)
    sig = compute_minhash_signature_fast(shingles)
    signatures_fast.append(sig)

end = time.time()
print(f"Time using XOR-based minhashing: {end - start:.2f} seconds")


Time using XOR-based minhashing: 25.49 seconds


When using my laptop, the code above runs the faster signature method and times it. This method took 497.42 seconds, or 8 minutes and 29 seconds. According to the above output, this method is faster than using 200 hashes. The time when being run on my pc (the above output), the time is 25.49 seconds. 

### 4. [10 points] Write a program to allocate the reviews among 216 buckets. 

In [8]:
NUM_BUCKETS = 2**16  # 65536

def signature_to_bucket(signature):
    """Hash a signature into one of 65,536 buckets."""
    sig_str = ''.join(map(str, signature))
    hash_val = int(hashlib.md5(sig_str.encode()).hexdigest(), 16)
    return hash_val % NUM_BUCKETS

# Reassign all reviews to one of 65536 buckets
buckets = [signature_to_bucket(sig) for sig in signatures_fast]


### 5. [10 points] How many non-empty buckets did you get?

In [9]:
used_buckets = set(buckets)
all_buckets = set(range(NUM_BUCKETS))

empty_buckets = all_buckets - used_buckets
non_empty = len(all_buckets - empty_buckets)
num_empty_buckets = len(empty_buckets)

print(f"Number of non-empty buckets (out of 65536): {non_empty}")


Number of non-empty buckets (out of 65536): 16537


According to the above output, there are 16,537 non-empty buckets. 

### 6. [10 points] Find actual similar reviews: Compute the exact Jaccard similarity for all pairs of reviews and output the pairs of reviews that have a similarity at least 0.5.

In [10]:
# Generate all shingle sets once
shingle_sets = [shingle_review(review) for review in df_filtered['Review Text']]

from itertools import combinations

similar_pairs_jaccard = []

for i, j in combinations(range(len(shingle_sets)), 2):
    s1, s2 = shingle_sets[i], shingle_sets[j]
    intersection = len(s1.intersection(s2))
    union = len(s1.union(s2))
    if union == 0:
        continue
    similarity = intersection / union
    if similarity >= 0.5:
        similar_pairs_jaccard.append((i, j, similarity))

print(f"Found {len(similar_pairs_jaccard)} pairs with Jaccard similarity >= 0.5")

for i, j, sim in similar_pairs_jaccard[:5]:
    print(f"Review {i} and Review {j} -> Jaccard: {sim:.3f}")

Found 11 pairs with Jaccard similarity >= 0.5
Review 1538 and Review 10295 -> Jaccard: 1.000
Review 2936 and Review 11684 -> Jaccard: 0.611
Review 3194 and Review 14506 -> Jaccard: 0.737
Review 3923 and Review 15080 -> Jaccard: 0.648
Review 5386 and Review 18642 -> Jaccard: 0.979


In [11]:
for i, j, sim in similar_pairs_jaccard:
    print(f"Review {i} and Review {j} -> Jaccard: {sim:.3f}")

Review 1538 and Review 10295 -> Jaccard: 1.000
Review 2936 and Review 11684 -> Jaccard: 0.611
Review 3194 and Review 14506 -> Jaccard: 0.737
Review 3923 and Review 15080 -> Jaccard: 0.648
Review 5386 and Review 18642 -> Jaccard: 0.979
Review 7741 and Review 17843 -> Jaccard: 1.000
Review 7835 and Review 11863 -> Jaccard: 0.647
Review 8330 and Review 11757 -> Jaccard: 1.000
Review 10200 and Review 17229 -> Jaccard: 0.577
Review 11249 and Review 17839 -> Jaccard: 0.842
Review 13923 and Review 17506 -> Jaccard: 1.000


As you can see from the above output, there were 11 exact Jaccard similarities found with this method. 

### 7. [15 points] Find similar reviews using min-hashing: Compute signatures for all reviews using 200 hash functions. Output the pairs that have a similarity of at least 0.5.

In [12]:
similar_pairs_minhash = []

for i, j in combinations(range(len(signatures)), 2):
    sig1, sig2 = signatures[i], signatures[j]
    matches = sum(1 for a, b in zip(sig1, sig2) if a == b)
    similarity = matches / NUM_HASHES
    if similarity >= 0.5:
        similar_pairs_minhash.append((i, j, similarity))

print(f"Found {len(similar_pairs_minhash)} pairs with MinHash similarity >= 0.5")


Found 418 pairs with MinHash similarity >= 0.5


In [ ]:
for i, j, sim in similar_pairs_minhash:
    print(f"Review {i} and Review {j} -> MinHash similarity: {sim:.3f}")


From the above output, there were 418 pairs with a MinHash similarity above 0.5. 

### 8. [10 points] Compare min-hashing to truth: Use your results from the previous two problems to determine the following
        The count of true positives: actual similar reviews identified by min-hashing as similar.
        The count of true negatives: actual dissimilar reviews identified by min-hashing as dissimilar.
        The count of false negatives: actual similar reviews identified by min-hashing as dissimilar.
        The count of false positives: actual dissimilar reviews identified by min-hashing as similar.


In [23]:
actual_similar = set((i, j) for i, j, _ in similar_pairs_jaccard)
approx_similar = set((i, j) for i, j, _ in similar_pairs_minhash)

# Ensure all pair indices are in the same order
all_pairs = set(combinations(range(len(shingle_sets)), 2))

In [24]:
TP = len(actual_similar & approx_similar)
FP = len(approx_similar - actual_similar)
FN = len(actual_similar - approx_similar)
TN = len(all_pairs - (actual_similar | approx_similar))

print(f"True Positives: {TP}")
print(f"False Positives: {FP}")
print(f"False Negatives: {FN}")
print(f"True Negatives: {TN}")


True Positives: 11
False Positives: 407
False Negatives: 0
True Negatives: 182586077


As you can see from above, there are only the 11 true positives from the earlier actual similarites. There were 407 false positives and zero false negatives. 

### 9. [5 points] Compare min-hashing to truth using the cheap hash functions outlined in question 3. 

In [25]:
similar_pairs_minhash_fast = []

for i, j in combinations(range(len(signatures_fast)), 2):
    sig1, sig2 = signatures_fast[i], signatures_fast[j]
    matches = sum(1 for a, b in zip(sig1, sig2) if a == b)
    similarity = matches / NUM_HASHES
    if similarity >= 0.5:
        similar_pairs_minhash_fast.append((i, j, similarity))

approx_similar_fast = set((i, j) for i, j, _ in similar_pairs_minhash_fast)


In [26]:
TP_fast = len(actual_similar & approx_similar_fast)
FP_fast = len(approx_similar_fast - actual_similar)
FN_fast = len(actual_similar - approx_similar_fast)
TN_fast = len(all_pairs - (actual_similar | approx_similar_fast))

print(f"Fast MinHashing:")
print(f"  True Positives: {TP_fast}")
print(f"  False Positives: {FP_fast}")
print(f"  False Negatives: {FN_fast}")
print(f"  True Negatives: {TN_fast}")


Fast MinHashing:
  True Positives: 11
  False Positives: 410
  False Negatives: 0
  True Negatives: 182586074


This method resulted in 3 more false positives with 410. However, this method is also much faster, so the drop in accuracy is likely worth the increase in performance. 

### 10. [5 points] Comment on the performance difference (TP, TN, FN, FP) between questions 8 and 9. 

In [27]:
print("=== Comparison ===")
print(f"Standard MinHashing   -> TP: {TP}, FP: {FP}, FN: {FN}, TN: {TN}")
print(f"Cheap Hash Functions  -> TP: {TP_fast}, FP: {FP_fast}, FN: {FN_fast}, TN: {TN_fast}")


=== Comparison ===
Standard MinHashing   -> TP: 11, FP: 407, FN: 0, TN: 182586077
Cheap Hash Functions  -> TP: 11, FP: 410, FN: 0, TN: 182586074


The cheap method has 3 more false positives, with the same amount of false negatives. With the cheap method being much faster, it is worth the small drop in accuracy for the increase in speed. 